In [1]:
import math as m
import numpy as np
import ipywidgets as wg
import matplotlib.pyplot as plt

# Function that Computes the Pattern for an Omnidirectional Antenna
def itu_omni_antenna(gain_peak, tilt, side_lobe_param, eirp):
    """
    Based on ITU-R Recommendation F.1336-5, published January 2019
    Reference pattern for omnidirectional base station antenna 
    Applicable frequency range: 400 MHz to 70 GHz
    Uses the method for representing peak side-lobe patterns (ref. 2.1)

    Parameters:
    gain_peak (float): Maximum antenna gain in dBi
    tilt (float): electrical tilt in degrees measured from the horizon
    side_lobe_param (float): increased sidelobe levels (ref. 2.3 and 2.4)
    eirp (float): peak EIRP

    Returns:
    Composite gain, heatmap, and elevation angles
    """
    itilt = -1*tilt
    azimuth = np.linspace(-180,180,361)
    theta_h = np.linspace(-90,90,181)
    theta_e = np.piecewise(theta_h,
                           [theta_h+itilt>=0,
                            theta_h+itilt<0],
                           [lambda theta_h: (90*(theta_h+itilt))/(90+itilt),
                            lambda theta_h: (90*(theta_h+itilt))/(90-itilt)]
                          )
    threedB_BW = 107.6*10**(-0.1*gain_peak)
    sidelobe_bump = threedB_BW*np.sqrt(1-((1/1.2)*np.log10(side_lobe_param+1)))
    gain_pattern = np.piecewise(theta_e,
                        [np.abs(theta_e)<sidelobe_bump,
                         (np.abs(theta_e)>=sidelobe_bump) & (np.abs(theta_e)<threedB_BW),
                         np.abs(theta_e)>=threedB_BW],
                        [lambda theta_e: gain_peak - 12*(theta_e/threedB_BW)**2,
                         lambda theta_e: gain_peak - 12 + 10*np.log10(side_lobe_param+1),
                         lambda theta_e: gain_peak - 12 + 10*np.log10((abs(theta_e)/threedB_BW)**-1.5 + side_lobe_param)]
                       )
    heat_pattern = np.tile(gain_pattern, (len(azimuth),1)).T
    heat_values = (eirp - gain_peak) + heat_pattern
    return gain_pattern, heat_values, theta_h

# Function that Computes the Pattern for a Directional Antenna
def itu_sectoral_antenna(G0, phi_3db, e_tilt, m_tilt, eirp, kp=0.7, kh=0.7, kv=0.3):
    """
    Baseed on ITU-R Recommendation F.1336-5 section 3.1, published January 2019
    Reference pattern for base station antennas with sectoral coverage
    Uses the method for representing peak side-lobe patterns (3.1.1)
    
    Parameters:
    phi (float or array): Azimuth angle in degrees
    theta (float or array): Elevation angle in degrees
    G0 (float): Maximum antenna gain in dBi
    phi_3db (float): Horizontal 3 dB beamwidth in degrees
    theta_3db (float): Vertical 3 dB beamwidth in degrees
    kp, kh, kv (float): Sidelobe adjustment factors (defaults in 3.1.1)

    Returns:
    Composite gain, horizontal gain, and vertical gain in dBi
    """
    # Generate angles
    phi_h = np.linspace(-180, 180, 361)
    theta_h = np.linspace(-90, 90, 181)
    phi_h_mesh, theta_h_mesh = np.meshgrid(phi_h, theta_h)
    phi_h_rad = np.radians(phi_h_mesh)
    theta_h_rad = np.radians(theta_h_mesh)
        
    # Apply electrical tilt
    etilt = -1*e_tilt
    etilt_rad = np.radians(etilt)
    theta_e = np.piecewise(theta_h_rad,
                           [theta_h_rad + etilt_rad >= 0,
                            theta_h_rad + etilt_rad < 0],
                           [lambda t: ((np.pi/2)*(t+etilt_rad))/((np.pi/2)+etilt_rad),
                            lambda t: ((np.pi/2)*(t+etilt_rad))/((np.pi/2)-etilt_rad)]
                          )

    # Apply mechanical tilt
    mtilt = -1*m_tilt
    mtilt_rad = np.radians(mtilt)
    sin_theta = np.sin(theta_e) * np.cos(mtilt_rad) + np.cos(theta_e) * np.cos(phi_h_rad) * np.sin(mtilt_rad)
    theta_m = np.arcsin(sin_theta)
    cos_phi = (-np.sin(theta_e) * np.sin(mtilt_rad) + np.cos(theta_e) * np.cos(phi_h_rad) * np.cos(mtilt_rad)) / np.cos(theta_m)
    phi_m = np.arccos(np.clip(cos_phi, -1, 1))  # Clip to avoid domain errors
    theta = np.rad2deg(theta_m)
    phi = np.rad2deg(phi_m)
    phi = np.where(phi_h_rad < 0, -phi, phi)
    
    # Compute the 3dB beamwidth in the elevation plane (3.3 eq. 3a)
    if phi_3db <= 120:
        theta_3db = (31000*10**(-0.1*G0))/phi_3db

    # Compute the relative minimum gain (3.1.1.1 eq. 2b1)
    G180 = -12 + 10*np.log10(1+8*kp) - 15*np.log10(180/theta_3db)

    # Compute the relative reference gain in the azimuth plane (3.1.1.2 eq. 2b2)
    xh = np.abs(phi)/phi_3db
    lambda_kh = 3*(1 -0.5**(-kh))
    Ghr_xh = np.piecewise(xh,
                           [xh <= 0.5,
                            xh > 0.5],
                           [lambda x: -12*(x**2),
                            lambda x: -12*(x**(2-kh))-lambda_kh]
                          )
    Ghr = np.where(Ghr_xh < G180, G180, Ghr_xh)
    
    # Compute the relative reference gain in the elevation plane (3.1.1.3 eq. 2b3)
    xv = np.abs(theta)/theta_3db
    xk = m.sqrt(1 - 0.36*kv)
    C_incline = 10*np.log10((((180/theta_3db)**1.5)*(4**(-1.5)+kv))/(1+8*kp))/np.log10(22.5/theta_3db)
    lambda_kv = 12 - C_incline*np.log10(4) - 10*np.log10(4**(-1.5)+kv)
    Gvr_xv = np.piecewise(xv,
                          [xv < xk,
                           (xv >= xk) & (xv < 4),
                           (xv >= 4) & (xv < (90/theta_3db)),
                           xv >= (90/theta_3db)],
                          [lambda x: -12*(x**2),
                           lambda x: -12 + 10*np.log10(x**(-1.5)+kv),
                           lambda x: -lambda_kv - C_incline*np.log10(x),
                           lambda x: G180]
                         )
    Gvr = Gvr_xv
    
    # Compute the horizontal gain compression ratio (3.1.1 eq. 2a2)
    horiz_BWratio = 180/phi_3db
    Ghr_az = -12*horiz_BWratio**2 if horiz_BWratio <= 0.5 else -12*horiz_BWratio**(2-kh)-lambda_kh
    Ghraz = max(Ghr_az,G180)
    Ghr_0 = -12*0**2
    Rcomp = (Ghr-Ghraz)/(Ghr_0-Ghraz)

    # Compute the composite pattern over all angles (3.1.1 eq. 2a1)
    gain = G0 + Ghr + Rcomp*Gvr
    heat = (eirp - G0) + gain

    return gain, Ghr, Gvr, heat, theta_3db


# Function that Creates the Output Display for Omnidirectional Patterns
def plot_omni(gain_peak, gain_nominal, gain, heat_nominal, heat, elevation, eirp):
    
    fig = plt.figure(figsize=(8, 16))
    
    # Cartesian Plot
    ax1 = fig.add_subplot(311)
    ax1.plot(gain_nominal, elevation, color='black', linewidth=1, linestyle=':', label='nominal gain')
    ax1.plot(gain, elevation, color='blue', linewidth=1, linestyle='-', label='with sidelobes and tilt')
    ax1.set_yticks(np.linspace(min(elevation), max(elevation), 13))
    ax1.set_ylabel('Elevation Angle (deg.)')
    ax1.set_xlabel('Gain (dBi)')
    ax1.set_title('ITU-R F.1336.5 Omnidirectional Antenna Gain')
    ax1.legend()
    ax1.grid(True)
    
    # Polar Plot
    theta_rad = np.radians(elevation)
    ax2 = fig.add_subplot(312, projection='polar')
    ax2.plot(theta_rad, gain_nominal, color='black', linewidth=1, linestyle=':', label='nominal gain')
    ax2.plot(theta_rad, gain, color='blue', linewidth=1, linestyle='-', label='with sidelobes and tilt')
    ax2.set_thetamin(-90)
    ax2.set_thetamax(90)
    ax2.set_thetagrids(np.arange(-90, 91, 30))
    ax2.set_rlim(-30, gain_peak)
    ax2.set_ylabel("Gain (dBi)", labelpad=-30)
    ax2.set_title("ITU-R F.1336.5 Omnidirectional Antenna Gain")
    ax2.legend(loc='upper right', fontsize=8)
    ax2.annotate(f'Peak: {gain_peak:.1f} dBi',
                 fontsize='9', color='black',
                 xy=(0, gain_peak), xytext=(0.88, 0.6),
                 textcoords='axes fraction', ha='center', va='top')
    
    # Heatmap
    ax3 = fig.add_subplot(313)
    im = ax3.imshow(heat, extent=[-180,180,-90,90],
                    aspect='auto', origin='lower',
                    cmap='jet', vmin=m.floor(np.min(heat_nominal)), vmax=eirp
                    )
    ax3.set_xlabel('Azimuth Angle (deg.)')
    ax3.set_ylabel('Elevation Angle (deg.)')
    ax3.set_title('Spatial Emissions Pattern w/ ITU-R F.1336.5 Omnidirectional Antenna')
    plt.colorbar(im, ax=ax3, label='EIRP (dBm/10MHz)')
 
    plt.tight_layout()
    plot_output = wg.Output()
    with plot_output:
        display(fig)
    plt.close(fig)
    return plot_output

# Function that Creates the Output Display for Directional Patterns
def plot_sector(gain_peak, gain, gain_horiz, gain_vert, heat, bw_vert, tilt_elec, tilt_mech, eirp):
    
    fig = plt.figure(figsize=(8, 32))
    
    # Polar plot of horizontal gain pattern
    phi = np.linspace(-180, 180, 361)
    theta = np.linspace(-90, 90, 181)
    phi_rad = np.radians(phi)
    threedb_h= np.full_like(phi_rad, -3)
    gain_horiz_plot = gain_horiz[len(theta)//2, :]
    ax1 = fig.add_subplot(411, projection='polar')
    ax1.plot(phi_rad, gain_horiz_plot, color='blue', linewidth=1, linestyle='-', label='horizontal gain')
    ax1.plot(phi_rad, threedb_h, color='black', linewidth=1, linestyle=':', label='-3dB reference')
    ax1.set_theta_zero_location('N') ## zero at the top
    ax1.set_theta_direction(-1)  ## clockwise
    ax1.set_thetamin(-180)
    ax1.set_thetamax(180)
    ax1.set_thetagrids(np.arange(-180, 180, 15))
    ax1.set_rlim(-30, gain_peak)
    # ax1.set_ylabel("Gain (dBi)", labelpad=-30)
    ax1.set_title("ITU-R F.1336.5 Directional Antenna Pattern")
    ax1.legend(loc='upper right', fontsize=8)
    ax1.annotate(f'Peak: {gain_peak:.1f} dBi',
                 fontsize='9', color='black',
                 xy=(0, 0), xytext=(0.88, 0.6),
                 textcoords='axes fraction', ha='center', va='top')
    
    # Polar plot of vertical gain pattern
    theta_rad = np.radians(theta)
    threedb_v = np.full_like(theta_rad, -3)
    gain_vert_plot = gain_vert[:, len(phi)//2]
    ax2 = fig.add_subplot(412, projection='polar')
    ax2.plot(theta_rad, gain_vert_plot, color='blue', linewidth=1, linestyle='-', label='elevation gain')
    ax2.plot(theta_rad, threedb_v, color='black', linewidth=1, linestyle=':', label='-3dB reference')
    ax2.set_thetamin(-90)
    ax2.set_thetamax(90)
    ax2.set_thetagrids(np.arange(-90, 91, 15))
    ax2.set_rlim(-30, gain_peak)
    ax2.set_ylabel("Gain (dBi)", labelpad=-30)
    ax2.set_title("ITU-R F.1336.5 Directional Antenna Pattern")
    ax2.legend(loc='upper right', fontsize=8)
    ax2.annotate(f'Beamwidth: {bw_vert:.1f} deg.',
                 fontsize='9', color='black',
                 xy=(0, 0), xytext=(0.95, 0.75),
                 textcoords='axes fraction', ha='right', va='top')
    ax2.annotate(f'Peak: {gain_peak:.1f} dBi',
                 fontsize='9', color='black',
                 xy=(0, 0), xytext=(0.95, 0.7),
                 textcoords='axes fraction', ha='right', va='top')
    
    # Heatmap
    tilt_sum = (tilt_elec + tilt_mech)
    ax4 = fig.add_subplot(413)
    # im = ax4.imshow(gain, extent=[-180,180,-90,90],
    #                 aspect='auto', origin='lower',
    #                 cmap='jet', vmin=m.floor(np.min(gain)), vmax=gain_peak
    #                 )
    im = ax4.imshow(heat, extent=[-180,180,-90,90],
                    aspect='auto', origin='lower',
                    cmap='jet', vmin=m.floor(np.min(heat)), vmax=eirp
                    )
    ax4.axhline(y=tilt_sum, color='white', linewidth=1, linestyle=':', label='total tilt')
    ax4.set_aspect(0.9)
    ax4.set_xlim(-180, 180)
    ax4.set_ylim(-90, 90)
    ax4_xticks = [-180, -150, -120, -90, -60, -30, 0, 30, 60, 90, 120, 150, 180]
    ax4.set_xticks(ax4_xticks)
    ax4.set_xticklabels(ax4_xticks)
    ax4_yticks = [-90, -75, -60, -45, -30, -15, 0, 15, 30, 45, 60, 75, 90]
    ax4.set_yticks(ax4_yticks)
    ax4.set_yticklabels(ax4_yticks)
    ax4.set_xlabel('Azimuth Angle (deg.)')
    ax4.set_ylabel('Elevation Angle (deg.)')
    ax4.set_title('Spatial Emissions Pattern w/ ITU-R F.1336.5 Directional Antenna')
    cbar_ax4 = fig.add_axes([ax4.get_position().x1 + 0.02,
                            ax4.get_position().y0,
                            0.02,
                            ax4.get_position().height])
    cbar4 = fig.colorbar(im, cax=cbar_ax4)
    # cbar4.set_label('Gain (dBi)')
    cbar4.set_label('EIRP (dBm/10MHz)')
    
    # 3D spherical plot
    phi2 = np.linspace(-np.pi, np.pi, 361)
    theta2 = np.linspace(0, np.pi, 181)
    PHI, THETA = np.meshgrid(phi2, theta2)
    GAIN = gain
    X = GAIN * np.sin(np.pi-THETA) * np.cos(PHI)
    Y = GAIN * np.sin(np.pi-THETA) * np.sin(PHI)
    Z = GAIN * np.cos(np.pi-THETA)
    ax3 = fig.add_subplot(414, projection='3d')
    ax3.view_init(azim=45, elev=30)
    norm3 = plt.Normalize(GAIN.min(), GAIN.max())
    sm3 = plt.cm.ScalarMappable(cmap='jet', norm=norm3)
    sm3.set_array([])
    surf3 = ax3.plot_surface(X, Y, Z, facecolors=plt.cm.jet(norm3(GAIN)), alpha=0.6)
    cbar3 = fig.colorbar(sm3, ax=ax3, shrink=0.5, aspect=5)
    cbar3.set_label('Directional Gain (dBi)')
    orig_x, orig_y, orig_z = np.meshgrid(np.array([0]), np.array([0]), np.array([0]))
    maxGain_loc = np.unravel_index(GAIN.argmax(), GAIN.shape)
    maxGain_index = tuple(int(i) for i in maxGain_loc)
    maxGain_phi = phi2[maxGain_index[1]]
    maxGain_theta = theta2[maxGain_index[0]]
    peak_x = GAIN.max() * np.sin(np.pi-maxGain_theta) * np.cos(maxGain_phi)
    peak_y = GAIN.max() * np.sin(np.pi-maxGain_theta) * np.sin(maxGain_phi)
    peak_z = GAIN.max() * np.cos(np.pi-maxGain_theta)
    tip_x, tip_y, tip_z = 1.25*peak_x, 1.25*peak_y, 1.25*peak_z
    ax3.quiver(orig_x, orig_y, orig_z, tip_x, tip_y, tip_z, arrow_length_ratio=0.1)
    ax3.set_xlabel('X')
    ax3.set_ylabel('Y')
    ax3.set_zlabel('Z')
    ax3.set_xticklabels([])
    ax3.set_yticklabels([])
    ax3.set_zticklabels([])
    ax3.set_title('ITU-R F.1336.5 Directional Antenna Pattern')
    ax3.set_box_aspect((1, 1, 1))
    
    plt.subplots_adjust(right=0.9)
    # plt.tight_layout()
    plot_output = wg.Output()
    with plot_output:
        display(fig)
    plt.close(fig)
    return plot_output

# Function that Updates the Patterns in the Output Display
def update_patterns(ant_type, gain_peak, horiz_bw, etilt, mtilt, side_lobe_param, eirp):
    if ant_type == 'Omni':
        gain_nominal, heat_nominal, elevation = itu_omni_antenna(gain_peak, 0, 0, eirp)
        gain, heat, elevation = itu_omni_antenna(gain_peak, etilt, side_lobe_param, eirp)
        plot_output = plot_omni(gain_peak, gain_nominal, gain, heat_nominal, heat, elevation, eirp)
    elif ant_type == 'Directional':
        gain, gain_horiz, gain_vert, heat, bw_vert = itu_sectoral_antenna(gain_peak, horiz_bw, etilt, mtilt, eirp)
        plot_output = plot_sector(gain_peak, gain, gain_horiz, gain_vert, heat, bw_vert, etilt, mtilt, eirp)
    
    return plot_output

# Update Function: Triggers the patterns to update when any of the inputs are changed
def update(*args):
    if power_select.value == 'EIRP':
        eirp = power_widget.value    
    elif power_select.value == 'Conducted Power':
        eirp = power_widget.value + gain_widget.value
    plot_output = update_patterns(type_widget.value,
                                  gain_widget.value,
                                  beamwidth_widget.value,
                                  etilt_widget.value,
                                  mtilt_widget.value,
                                  sidelobe_widget.value,
                                  eirp
                                 )
    patterns.clear_output(wait=True)
    with patterns:
        display(plot_output)

## INPUTS AND OUTPUTS

# Create Input Widgets
styl = {'description_width':'initial'}
box1 = wg.Layout(width='150px')
box2 = wg.Layout(width='220px')
box3 = wg.Layout(width='150px')
heading_type = wg.HTML(value="<b>Antenna Type</b>")
heading_ant = wg.HTML(value="<b>Antenna Attributes</b>")
heading_tx = wg.HTML(value="<b>Transmitter Attributes</b>")
type_widget = wg.RadioButtons(options=['Omni', 'Directional'],value='Omni',layout=box1)
gain_widget = wg.BoundedFloatText(description='Peak Antenna Gain (dBi):',value=8.5,min=0,max=25,step=0.5,style=styl,layout=box2)
beamwidth_widget = wg.BoundedFloatText(description='Horizontal Beamwidth (deg.):',value=65,min=10,max=90,step=1,style=styl,layout=box2)
etilt_widget = wg.BoundedFloatText(description='Electrical Tilt (degrees):',value=0,min=-10,max=10,step=0.5,style=styl,layout=box2)
mtilt_widget = wg.BoundedFloatText(description='Mechanical Tilt (degrees):',value=0,min=-10,max=10,step=0.5,style=styl,layout=box2)
sidelobe_widget = wg.BoundedFloatText(description='Sidelobe Parameter (k):',value=0.7,min=0,max=5,step=0.1,style=styl,layout=box2)
power_select = wg.RadioButtons(options=['EIRP','Conducted Power'],value='EIRP',layout=box3)
power_widget = wg.FloatText(description='(dBm/10MHz)',value=47,layout=box3)
placeholder = wg.HTML("")
stack3 = wg.Stack([etilt_widget, beamwidth_widget], selected_index=0)
stack4 = wg.Stack([sidelobe_widget, etilt_widget], selected_index=0)
stack5 = wg.Stack([placeholder, mtilt_widget], selected_index=0)
wg.jslink((type_widget,'index'),(stack3,'selected_index'))
wg.jslink((type_widget,'index'),(stack4,'selected_index'))
wg.jslink((type_widget,'index'),(stack5,'selected_index'))

# Monitor Widgets for Changes to User Input
type_widget.observe(update,names='value')
gain_widget.observe(update,names='value')
beamwidth_widget.observe(update,names='value')
etilt_widget.observe(update,names='value')
mtilt_widget.observe(update,names='value')
sidelobe_widget.observe(update,names='value')
power_select.observe(update,names='value')
power_widget.observe(update,names='value')

# Display the Widgets
type_selection = wg.VBox([heading_type, type_widget])
antenna_parameters = wg.VBox([heading_ant, gain_widget, stack3, stack4, stack5])
transmitter_parameters = wg.VBox([heading_tx, power_select, power_widget])
grid = wg.GridBox([type_selection, antenna_parameters, transmitter_parameters],
                  layout = wg.Layout(
                      width = '700px',
                      grid_template_columns = '150px auto auto',
                      grid_template_rows = 'auto',
                      grid_gap = '10px'
                  )
                 )
display(grid)

# Display the Patterns
patterns = wg.Output()
display(patterns)

update()

GridBox(children=(VBox(children=(HTML(value='<b>Antenna Type</b>'), RadioButtons(layout=Layout(width='150px'),…

Output()